In [0]:
from google.colab import files
files.upload()

In [0]:
import numpy as np
import keras
from keras.layers import BatchNormalization, Conv2D, Reshape, Flatten, Activation, Dense
from keras.models import Sequential
from sudoku_data import getSudokuData, normalize, denormalize

Using TensorFlow backend.


In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d bryanpark/sudoku

 91% 62.0M/68.1M [00:00<00:00, 56.3MB/s]
100% 68.1M/68.1M [00:00<00:00, 87.5MB/s]


Obtenemos el conjunto de datos desde Kaggle, luego los preprocesamos en el archivo adjunto para obtener nuestros pares de conjuntos para las pruebas y el entrenamiento

In [0]:
(X_train, y_train),(X_test,y_test) = getSudokuData('sudoku.zip')

Al igual que en la identificación de números, se usará una red neural convolucional de 3 capas de convolución donde la primera tiene la dimensión de un tablero de SUDOKU 9x9x1 y entre capa y capa, se normalizan las salidas para evitar los números exageradamente grandes. 

Finalmente se aplanan para ir a la etapa de clasificación donde la habrá 9x9x9 neuronas, las cuales representan las 81 casillas y las 9 posibilidades para cada casilla.

Una vez teniendo las 729 probabilidades se agrupan a cada una de las probabilidades a una casilla,quedando como resultado un tensor de 81x9 donde se aplicará softmax

In [0]:
model = Sequential() 
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same')) 
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((81, 9)))
model.add(Activation('softmax')) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.adam(lr=.001))  

In [8]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 800000 samples, validate on 200000 samples
Epoch 1/2
800000/800000 [==============================] - 225s 281us/step - loss: 0.4913 - val_loss: 0.3726
Epoch 2/2
800000/800000 [==============================] - 217s 272us/step - loss: 0.3570 - val_loss: 0.3514


In [9]:
print(np.argmax(model.predict(X_test[0].reshape(1,9,9,1)).squeeze(), axis=1).reshape((9,9))+1)
print(model.predict(X_test[0].reshape(1,9,9,1)).shape)
print(y_test[0].reshape(9,9)+1)

[[7 1 2 3 9 5 4 8 6]
 [6 3 5 1 4 2 4 9 7]
 [9 8 4 3 3 6 5 9 2]
 [5 2 6 9 7 1 5 4 8]
 [8 4 3 5 6 2 1 7 9]
 [1 7 9 3 4 8 6 2 5]
 [3 6 8 2 5 9 7 5 4]
 [2 5 7 4 1 7 9 6 3]
 [4 9 7 6 5 3 2 5 1]]
(1, 81, 9)
[[7 1 2 8 9 5 4 3 6]
 [6 3 5 1 2 4 8 9 7]
 [9 8 4 7 3 6 5 1 2]
 [5 2 6 9 7 1 3 4 8]
 [8 4 3 5 6 2 1 7 9]
 [1 7 9 3 4 8 6 2 5]
 [3 6 8 2 1 9 7 5 4]
 [2 5 1 4 8 7 9 6 3]
 [4 9 7 6 5 3 2 8 1]]


Como se quiere recibir un sudoku, solo se debe predecir un juego, por lo que se envía un juego de dimensión 9x9x1 que hay que redimensionar a 1x9x9x1 ya que predict funciona con varias muestras.

El resultado predicho tiene dimensión 1x81x9 por lo que usamos squeeze para quitar la dimensión 1 y dejarlo en 81x9.

Luego se aplica np.argmax para obtener el índice con mayor probabilidad entre las 9 posibilidades que tiene cada casilla, obteniendo un tensor de dimensión 81 que llamamos ax que lo redimensionamos a un tablero 9x9 pred y como las neuronas de salida representan los números posibles le sumamos 1.

Adicionalmente vamos a obtener un tensor con las probabilidades de cada uno de los valores de pred, para ello se busca la máxima probabilidad entre las 9 posibles para cada una de las 81 casillas con np.max y lo redimensionamos a un tablero de 9x9 que llamamos prob, cuyas probabilidades las redondeamos a 2 decimales.

Como los resultados están normalizados debido al preprocesado ideal para el entrenamiento y predicción, se desnormalizan para tener el resultado real del tablero. Hasta aquí serviría para obtener un juego con IA, pero lo ideal sería que llenara cuadro a cuadro para así aumentar sus probabilidades de no fallar.

En mask se van a evaluar las jugadas faltantes en el tablero del SUDOKU, se guarda 1 en la casilla donde falta colocar un número y 0 en donde ya está colocado.

Si la suma de casillas vacías es 0, termina el ciclo, de lo contrario va a multiplicar la matriz de probabilidades por la máscara, siendo 0 en las casillas donde ya hay colocado un número y siendo la probabilidad de prob en las casillas donde hay que poner un número.

Con np.argmax se ubica de entre las 81 casillas, la que tiene las mayor probabilidad (recordemos que las que ya están marcadas quedaron en 0 en el paso anterior), como argmax devuelve un índice, se utiliza unravel_index que devuelve la posición i,j de la casilla.

Ya con la posición i,j se toma el valor de pred, y se introduce en el sudoku.

Finalmente se noramaliza de nuevo el sudoku para repetir el proceso hasta que no queden casillas vacías.

In [0]:
def resolveSudoku(sudoku):
  sudoku = sudoku.reshape((9,9))
  while(1):
    out = model.predict(sudoku.reshape((1,9,9,1)))
    out = out.squeeze()
    ax = np.argmax(out, axis=1)
    pred = ax.reshape((9,9))+1 
    ax = np.max(out, axis=1)
    prob = np.around(ax.reshape((9,9)), 2) 
    sudoku = denormalize(sudoku)

    mask = (sudoku==0)

    if(mask.sum()==0):
        break
        
    real_prob = prob*mask

    index = np.argmax(real_prob)

    i, j = np.unravel_index(index, pred.shape)
    sudoku[i][j] = pred[i][j]
    sudoku = normalize(sudoku)
  
  return pred

A continuación se procede a realizar 1000 juegos de validación

In [58]:
samples = 1000
mistakes = 0
for i in range(samples):
  mistakes += not np.array_equal(resolveSudoku(X_test[i]),(y_test[i].reshape(9,9)+1))

print("Ejemplos: ", samples, "\nErrores: ", mistakes, "\nPorcentaje de acierto: ", (samples-mistakes)/samples)

Ejemplos:  1000 
Errores:  35 
Porcentaje de acierto:  0.965


Un caso de prueba

In [51]:
print(denormalize(X_test[0]).reshape(9,9))
print(resolveSudoku(X_test[0]))
print(y_test[0].reshape(9,9)+1)

[[0. 0. 2. 0. 9. 5. 0. 0. 0.]
 [6. 3. 0. 1. 0. 0. 0. 0. 0.]
 [0. 8. 4. 0. 0. 6. 0. 0. 2.]
 [0. 0. 0. 0. 7. 0. 0. 4. 8.]
 [0. 4. 0. 5. 0. 0. 1. 7. 0.]
 [1. 0. 9. 0. 0. 8. 6. 2. 0.]
 [3. 0. 8. 0. 0. 9. 7. 0. 0.]
 [2. 5. 0. 4. 0. 0. 9. 6. 0.]
 [0. 0. 0. 6. 0. 3. 0. 0. 1.]]
[[7 1 2 8 9 5 4 3 6]
 [6 3 5 1 2 4 8 9 7]
 [9 8 4 7 3 6 5 1 2]
 [5 2 6 9 7 1 3 4 8]
 [8 4 3 5 6 2 1 7 9]
 [1 7 9 3 4 8 6 2 5]
 [3 6 8 2 1 9 7 5 4]
 [2 5 1 4 8 7 9 6 3]
 [4 9 7 6 5 3 2 8 1]]
[[7 1 2 8 9 5 4 3 6]
 [6 3 5 1 2 4 8 9 7]
 [9 8 4 7 3 6 5 1 2]
 [5 2 6 9 7 1 3 4 8]
 [8 4 3 5 6 2 1 7 9]
 [1 7 9 3 4 8 6 2 5]
 [3 6 8 2 1 9 7 5 4]
 [2 5 1 4 8 7 9 6 3]
 [4 9 7 6 5 3 2 8 1]]
